In [1]:
import pandas as pd
import numpy as np

import requests
import json
from bs4 import BeautifulSoup

import string
import re
import pprint
import unidecode

import time

In [2]:
import capstone_functions as f

In [3]:
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

# Testing web-scrape code to make sure it works

In [22]:
trial = 'https://stackoverflow.com/jobs/406677/senior-platform-engineer-paigeai?so=i&pg=2&offset=0'
trial_response = requests.get(trial, headers = header)
trial_soup = BeautifulSoup(trial_response.text, 'html.parser')

In [23]:
tech = []
job_tech = trial_soup.find_all('section', {'class':'mb32'})[1]('a')
for x in job_tech:
    tech.append(x.text)
    
# tech

In [24]:
overview = ''
job_overview = trial_soup.find_all('section', {'class':'mb32'})[2](['p', 'ul'])
for y in job_overview:
    overview += y.text

# overview

In [26]:
position = str(trial_soup.find_all('h1', {'class':'fs-headline1 mb4'})[0].text)
# position

In [27]:
about = [trial_soup.find('section', {'class':'mb32'})('div')[1].text]
# about

---

# Loop functions to scrape website

In [4]:
# create an empty list to store all hyperlinks
all_hyperlinks = []

In [3]:
import capstone_functions as f

In [6]:
# loop to go through every page and get a link for each job posting
# at the time of the web scrape, there were 116 pages
i = 1
while i <= 116:
    url = 'https://stackoverflow.com/jobs?pg={}'.format(i)        
    f.get_hyperlinks(url, header, all_hyperlinks)
    i += 1

In [7]:
len(set(all_hyperlinks))

2845

In [8]:
len(all_hyperlinks)

2845

In [ ]:
all_hyperlinks[0]

In [9]:
job_desc_list = []

In [11]:
# loop for every url in list
# set up counter to check for errors
s = 0
for link in all_hyperlinks:
    url = link
    f.each_job(url, header, job_desc_list)
    s += 1
s

In [13]:
job_desc_list[0]

{'position': ['\nEngineering Manager '],
 'description': ['\n\nJob type: \nFull-time\n\n\nExperience level: \nManager\n\n\nRole: \nFull Stack Developer\n\n'],
 'languages': ['python',
  'django',
  'javascript',
  'react',
  'amazon-web-services'],
 'overview': "Force is reimagining the way patients recover from injury and surgery. We are partnered with leading health systems across the United States to deliver prescriptive plans of care to their patients, measure multiple clinical data points and provide actionable insights into care and recovery. We have a direct and immediate feedback loop to our end users (patients and providers) and pride ourselves in being agile and responsive.What you’ll be doing: Our ideal engineering manager has a background and knowledge of functioning as an individual technical contributor working with web and mobile technologies. As a manager, you’ll work with engineers to make good technical decisions and support the career growth and development of the te

In [88]:
# loop to add url of job posting to each dict
for idx, value in enumerate(all_hyperlinks):
    job_desc_list[idx]['url'] = value

In [90]:
job_desc_list[0]

{'position': ['\nSoftware Engineer '],
 'description': ['\n\nJob type: \nFull-time\n\n\nRole: \nBackend Developer\n\n\nIndustry: \nFinancial Services\n\n'],
 'languages': ['java', 'api', 'python'],
 'overview': "This role requires a wide variety of strengths and capabilities, including:     * BS/BA degree in Computer Science or AI/Machine Learning     * Provide technical expertise throughout the software lifecycle including design, implementation and delivery     * Understanding of cloud, virtualization, APIs, and modern software languages     * Software development experience in one or more general-purpose programming languages: Python, Java, C, C++, Go     * Experience with developing frameworks that help increase developer and release velocity, improving code health and technical standards     * Advanced experience working with two or more from the following: web application development, database, unix/linux environments, distributed and parallel systems, information retrieval, netw

In [146]:
# creating df of the list of dictionaries
df = pd.DataFrame(job_desc_list)

In [163]:
# dropped any rows with nan values for languages since this will be my target variable
df.dropna(subset=['languages'], inplace=True)

In [176]:
df.shape

(2891, 4)

In [243]:
# column had empty lists as values, so i removed those as well
final = df[df.astype(str)['languages'] != '[]'].copy()

In [244]:
final.shape

(2366, 4)

# Cleaning df with regex

In [266]:
# creating regex condition for cleaning punctuation
remove = string.punctuation
remove = remove.replace('/', '') # don't remove slash
pattern = r'[{}]'.format(remove)

https://stackoverflow.com/questions/21209024/python-regex-remove-all-punctuation-except-hyphen-for-unicode-string/37708340

In [469]:
clean = lambda x: f.clean_text(x, pattern)

In [470]:
final.position = pd.DataFrame(final.position.apply(clean))

In [471]:
final.description = pd.DataFrame(final.description.apply(clean))

In [472]:
final.overview = pd.DataFrame(final.overview.apply(clean))

In [546]:
final['clean_description'] = final.description.apply(lambda x: f.fix_description(x))

In [545]:
final['position_title'] = final.position.apply(lambda x: f.keep_position_name(x))

In [548]:
# dropping for cleaned columns
final.drop('position', axis=1, inplace=True)
final.drop('description', axis=1, inplace=True)

In [640]:
final.head()

,languages,overview,position_title,clean_description,target
1,"[amazon-web-services, python, docker, kubernet...",Paige is a software company helping pathologis...,Senior Software Engineer Backend Cloud Computing,Job type Full time Experience level Senior Rol...,1
3,"[python, javascript, reactjs, django, typescript]",About UsSimon Data was founded in by a team of...,Engineering Manager,Job type Full time Experience level Senior Man...,1
4,"[node.js, kubernetes, microservices, typescrip...",Who is MANTL MANTL is a fast growing NYC based...,Senior Backend Engineer NYC Or Remote,Job type Full time Experience level Mid Level ...,0
5,"[api, postgresql, scala, amazon-web-services, ...",As a SIMON Backend Engineer you will play an i...,Backend Engineer,Job type Full time Experience level Mid Level ...,0
6,"[go, ruby, ruby-on-rails, redis, postgresql]",Gravity is looking for passionate software eng...,Software Engineer Backend NYC/LA,Job type Full time Experience level Junior Mid...,0


# Creating target variable column

In [629]:
# created column for boolean result if 'python' appears in languages column
final['almost'] = final['languages'].apply(lambda x: any(['python' in x for value in final['languages']]))

In [637]:
# created target variable column
targ_dict = {True:1, False:0}
final['target'] = final['almost'].map(targ_dict)

In [639]:
# dropping almost column now that i have target column
final.drop('almost', axis=1, inplace=True)

In [644]:
final.head()

,languages,overview,position_title,clean_description,target
1,"[amazon-web-services, python, docker, kubernet...",Paige is a software company helping pathologis...,Senior Software Engineer Backend Cloud Computing,Job type Full time Experience level Senior Rol...,1
3,"[python, javascript, reactjs, django, typescript]",About UsSimon Data was founded in by a team of...,Engineering Manager,Job type Full time Experience level Senior Man...,1
4,"[node.js, kubernetes, microservices, typescrip...",Who is MANTL MANTL is a fast growing NYC based...,Senior Backend Engineer NYC Or Remote,Job type Full time Experience level Mid Level ...,0
5,"[api, postgresql, scala, amazon-web-services, ...",As a SIMON Backend Engineer you will play an i...,Backend Engineer,Job type Full time Experience level Mid Level ...,0
6,"[go, ruby, ruby-on-rails, redis, postgresql]",Gravity is looking for passionate software eng...,Software Engineer Backend NYC/LA,Job type Full time Experience level Junior Mid...,0


In [646]:
# saving dataframe to csv
# final.to_csv('job_post_dataframe.csv')